In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
weather_set = pd.read_csv("data/weather_burbank_airport.csv")

In [3]:
weather_set

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0


In [4]:
# check for duplicates
duplicate_counter = weather_set.duplicated().sum()
# duplicates = weather_set[weather_set.duplicated()]
print("sum of duplicates:", duplicate_counter)

sum of duplicates: 0


In [5]:
# check for none-values
nan_counts = weather_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: windspeed, Count of NaN-Values: 86
Feature: felt_temperature, Count of NaN-Values: 26
Feature: temperature, Count of NaN-Values: 25
Feature: cloud_cover_description, Count of NaN-Values: 20
Feature: cloud_cover, Count of NaN-Values: 20
Feature: pressure, Count of NaN-Values: 8
Feature: city, Count of NaN-Values: 0
Feature: timestamp, Count of NaN-Values: 0
Feature: precipitation, Count of NaN-Values: 0


-> Idee: NaN Werte mit dem Durchschnitt oder "Randoms" die den Durchschnitt ergeben auffüllen -> verlieren keine Rows
-> Ich würde alle Na-rows rausschmeißen, es handelt sich hier bei um 139 rows von insgesamt knapp 30.000, außerdem war das Hauptargument bei dem charging_set, dass wir die Na-Werte auffüllen bei den userInputs damit wir den rest der spalten nicht verlieren, da wir die gesondert in einer Analyse verwenden müssen. Das haben wir hier auch nicht zwingend. Wir haben mehr als genug datenpunkte um ein modell zu trainieren auch ohne die Na-rows, daher würde Ich die einfach rausschmeißen um das ergebnis des modells nicht zu beeinflussen mit "falschen" Werten

In [6]:
# remove missing data
weather_set.dropna()

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0


In [7]:
weather_set.dtypes

city                        object
timestamp                   object
temperature                float64
cloud_cover                float64
cloud_cover_description     object
pressure                   float64
windspeed                  float64
precipitation              float64
felt_temperature           float64
dtype: object

# Outlier detection

In [8]:
# Check ranges of numeric columns
numeric_columns = weather_set.select_dtypes(include=[np.number])
for col in numeric_columns.columns:
    valid_values = numeric_columns.loc[numeric_columns[col] != -1, col]  # exclude -1, since we used it as a placeholder for NaN-values
    print(f"Column '{col}': min={valid_values.min()}, max={valid_values.max()}")

# Check unique values of categorical columns
categorical_columns = weather_set.select_dtypes(exclude=[np.number])
for col in categorical_columns.columns:
    print(f"Column '{col}': unique values={categorical_columns[col].unique()}")

Column 'temperature': min=2.0, max=46.0
Column 'cloud_cover': min=4.0, max=47.0
Column 'pressure': min=971.0, max=999.65
Column 'windspeed': min=0.0, max=57.0
Column 'precipitation': min=0.0, max=18.54
Column 'felt_temperature': min=0.0, max=42.0
Column 'city': unique values=['Burbank']
Column 'timestamp': unique values=['2018-01-01 08:53:00' '2018-01-01 09:53:00' '2018-01-01 10:53:00' ...
 '2021-01-01 05:53:00' '2021-01-01 06:53:00' '2021-01-01 07:53:00']
Column 'cloud_cover_description': unique values=['Fair' 'Haze' 'Partly Cloudy' 'Mostly Cloudy' 'Cloudy' 'Fog' 'Light Rain'
 'Rain' 'Heavy Rain' 'Heavy Rain / Windy' 'Light Rain / Windy' 'T-Storm'
 'Fair / Windy' 'Cloudy / Windy' 'Mostly Cloudy / Windy'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'Thunder' nan 'Smoke'
 'Light Rain with Thunder' 'Heavy T-Storm' 'Rain / Windy' 'Blowing Dust']


# Feature Engineering

In [9]:
weather_set['temperature_difference'] = abs(weather_set['felt_temperature'] - weather_set['temperature'])

In [10]:
for column in weather_set.columns:
    unique_values = weather_set[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'city': ['Burbank']
Unique values in column 'timestamp': ['2018-01-01 08:53:00' '2018-01-01 09:53:00' '2018-01-01 10:53:00' ...
 '2021-01-01 05:53:00' '2021-01-01 06:53:00' '2021-01-01 07:53:00']
Unique values in column 'temperature': [ 9.  8.  7. 12. 16. 19. 21. 22. 23. 17. 15. 14. 13. 18. 24. 27. 25. 20.
 11. 10. 26. 28.  6.  4. 29. 30.  5.  3.  2. 31. 33. 34. 32. nan 36. 37.
 42. 43. 44. 45. 41. 39. 38. 40. 35. 46.]
Unique values in column 'cloud_cover': [33. 21. 29. 30. 34. 28. 26. 27. 20. 11. 12. 40.  4. 38. nan 47. 22. 19.]
Unique values in column 'cloud_cover_description': ['Fair' 'Haze' 'Partly Cloudy' 'Mostly Cloudy' 'Cloudy' 'Fog' 'Light Rain'
 'Rain' 'Heavy Rain' 'Heavy Rain / Windy' 'Light Rain / Windy' 'T-Storm'
 'Fair / Windy' 'Cloudy / Windy' 'Mostly Cloudy / Windy'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'Thunder' nan 'Smoke'
 'Light Rain with Thunder' 'Heavy T-Storm' 'Rain / Windy' 'Blowing Dust']
Unique values in column 'pressure': [

In [11]:
frequency = weather_set['cloud_cover_description'].value_counts(dropna=False)

print(frequency)

cloud_cover_description
Fair                       17122
Cloudy                      4936
Partly Cloudy               2668
Mostly Cloudy               1830
Light Rain                   896
Haze                         579
Smoke                        329
Fog                          325
Rain                         247
Heavy Rain                   120
Fair / Windy                  74
NaN                           20
T-Storm                       18
Thunder in the Vicinity       17
Partly Cloudy / Windy         14
Mostly Cloudy / Windy         10
Light Rain / Windy            10
Cloudy / Windy                 9
Heavy Rain / Windy             7
Blowing Dust                   5
Heavy T-Storm                  4
Rain / Windy                   2
Thunder                        1
Light Rain with Thunder        1
Name: count, dtype: int64


Feature Engineering:
- Konzept überlegen, mit dem man Überschneidungen zuteilt
- Idee: "Partly" "Light" Heavy" als eigenes Feature implementieren und die Einträge überarbeiten

WICHTIG: Spalten mit unnötigen Wörtern händisch umbennen

In [12]:
weather_set['cloud_cover_description'] = weather_set['cloud_cover_description'].fillna('')

# Definieren der Sonderfälle und deren neue Bezeichnungen
special_cases = {
    'Thunder in the Vicinity': 'thunder_in_vicinity',
    'Light Rain with Thunder': 'light_rain_thunder',
    'Blowing Dust': 'blowing_dust'
}

# Liste von Wörtern, die ignoriert werden sollen
stop_words = {'in', 'the'}

# Set für alle einzigartigen Features
unique_features = set()

for entry in weather_set['cloud_cover_description']:
    # Sonderfälle zuerst ersetzen
    for case, replacement in special_cases.items():
        if case in entry:
            entry = entry.replace(case, replacement)
    
    # Trennen bei Leerzeichen und Slashes
    words = entry.replace('/', ' ').split()
    
    # Wörter zu den Features hinzufügen, außer Stoppwörter
    unique_features.update(word for word in words if word.lower() not in stop_words)

# Für jedes Feature eine neue Spalte erstellen
for feature in unique_features:
    weather_set[feature.lower()] = weather_set['cloud_cover_description'].apply(
        lambda x: 1 if feature in x else 0
    )

In [13]:
weather_set

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature,temperature_difference,...,blowing_dust,fog,windy,light_rain_thunder,haze,thunder,mostly,thunder_in_vicinity,light,partly
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0,0.0,...,0,0,0,0,0,0,0,0,0,0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0,0.0,...,0,0,0,0,0,0,0,0,0,0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0,0.0,...,0,0,0,0,0,0,0,0,0,0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
weather_set.columns

Index(['city', 'timestamp', 'temperature', 'cloud_cover',
       'cloud_cover_description', 'pressure', 'windspeed', 'precipitation',
       'felt_temperature', 'temperature_difference', 'heavy', 't-storm',
       'cloudy', 'rain', 'smoke', 'fair', 'blowing_dust', 'fog', 'windy',
       'light_rain_thunder', 'haze', 'thunder', 'mostly',
       'thunder_in_vicinity', 'light', 'partly'],
      dtype='object')

In [15]:
random_entry = weather_set.sample(n=1).iloc[0]

# Zellenwerte einzeln ausgeben
for column_name, value in random_entry.items():
    print(f"{column_name}: {value}")

city: Burbank
timestamp: 2019-07-11 19:53:00
temperature: 32.0
cloud_cover: 34.0
cloud_cover_description: Fair
pressure: 986.48
windspeed: 0.0
precipitation: 0.0
felt_temperature: 31.0
temperature_difference: 1.0
heavy: 0
t-storm: 0
cloudy: 0
rain: 0
smoke: 0
fair: 1
blowing_dust: 0
fog: 0
windy: 0
light_rain_thunder: 0
haze: 0
thunder: 0
mostly: 0
thunder_in_vicinity: 0
light: 0
partly: 0


In [16]:
# weather_set['timestamp'] = pd.to_datetime(weather_set['timestamp_date'])

# weather_set['timestamp_year'] = charging_set['connectionTime_date'].dt.year
# charging_set['timestamp_month'] = charging_set['connectionTime_date'].dt.month

In [17]:
# # split time-related data into year, month, day, time
# ### split connectionTime in separate date and time columns
# # Convert the column to datetime format
weather_set['timestamp'] = pd.to_datetime(weather_set['timestamp'])

weather_set['timestamp_year'] = weather_set['timestamp'].dt.year
weather_set['timestamp_month'] = weather_set['timestamp'].dt.month
weather_set['timestamp_day'] = weather_set['timestamp'].dt.day
weather_set['timestamp_time'] = weather_set['timestamp'].dt.time

In [18]:
# suggestion: season, isWeekday, time instead
weather_set['timestamp'] = pd.to_datetime(weather_set['timestamp'])

# 0: january, february, march = spring
# 1: april, may, june = summer
# 2: july, august, september = fall
# 3: oktober, november, december = winter 
weather_set['season'] = (weather_set['timestamp'].dt.month - 1) // 3
weather_set['isWeekday'] = weather_set['timestamp'].dt.dayofweek.apply(lambda x: 1 if x > 5 else 0)
weather_set['timestamp_time'] = weather_set['timestamp'].dt.time

In [19]:
weather_set

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature,temperature_difference,...,mostly,thunder_in_vicinity,light,partly,timestamp_year,timestamp_month,timestamp_day,timestamp_time,season,isWeekday
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0,1.0,...,0,0,0,0,2018,1,1,08:53:00,0,0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0,0.0,...,0,0,0,0,2018,1,1,09:53:00,0,0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0,0.0,...,0,0,0,0,2018,1,1,10:53:00,0,0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0,0.0,...,0,0,0,1,2018,1,1,11:53:00,0,0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0,0.0,...,0,0,0,0,2018,1,1,12:53:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0,0.0,...,0,0,0,0,2021,1,1,03:53:00,0,0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0,0.0,...,0,0,0,0,2021,1,1,04:53:00,0,0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0,0.0,...,0,0,0,0,2021,1,1,05:53:00,0,0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0,0.0,...,0,0,0,0,2021,1,1,06:53:00,0,0


In [20]:
# weather_set.to_csv('data/cleaned_weather_set.csv', mode='w', index=False)

--- 

### Additional Weather Set

weather from 2021 is missing so we searched for it in the internet and found constructed a dataset of every week in 2021. 

In [21]:
weather_2021 = pd.read_csv("data/weather_2021.csv")

In [27]:
weather_2021.describe()

,temperature,dwpt,rhum,precipitation,snow,wdir,windspeed,wpgt,pressure,tsun,coco
count,8760.000000,8760.000000,8760.000000,8760.000000,0.0,8760.000000,8760.000000,0.0,8760.000000,0.0,668.000000
mean,18.272705,6.597740,53.062671,0.035434,NaN,130.623402,8.754144,NaN,1013.983402,NaN,5.196108
std,6.430203,7.300972,22.769755,0.374105,NaN,96.281520,6.946924,NaN,3.826846,NaN,0.665390
min,3.300000,-27.700000,2.000000,0.000000,NaN,0.000000,0.000000,NaN,1001.600000,NaN,5.000000
25%,13.900000,2.300000,35.000000,0.000000,NaN,10.000000,5.400000,NaN,1011.300000,NaN,5.000000
50%,17.800000,8.400000,56.000000,0.000000,NaN,140.000000,7.600000,NaN,1013.700000,NaN,5.000000
75%,22.200000,12.200000,73.000000,0.000000,NaN,180.000000,13.000000,NaN,1016.600000,NaN,5.000000
max,40.000000,17.900000,100.000000,15.500000,NaN,360.000000,64.800000,NaN,1026.100000,NaN,9.000000


In [28]:
weather_set.describe()

,timestamp,temperature,cloud_cover,pressure,windspeed,precipitation,felt_temperature,temperature_difference,heavy,t-storm,...,thunder,mostly,thunder_in_vicinity,light,partly,timestamp_year,timestamp_month,timestamp_day,season,isWeekday
count,29244,29219.000000,29224.000000,29236.000000,29158.000000,29244.000000,29218.000000,29218.000000,29244.000000,29244.000000,...,29244.000000,29244.000000,29244.0,29244.000000,29244.000000,29244.000000,29244.000000,29244.000000,29244.000000,29244.000000
mean,2019-06-29 23:51:42.252770048,17.885622,30.107993,986.879400,8.566568,0.059360,17.731433,0.197276,0.004480,0.000752,...,0.000650,0.062919,0.0,0.031015,0.091711,2018.998735,6.449870,15.707017,1.480646,0.141841
min,2018-01-01 08:53:00,2.000000,4.000000,971.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,2018.000000,1.000000,1.000000,0.000000,0.000000
25%,2018-10-06 05:38:00,13.000000,26.000000,984.500000,6.000000,0.000000,13.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,2018.000000,3.000000,8.000000,0.000000,0.000000
50%,2019-06-24 00:23:00,17.000000,33.000000,986.480000,7.000000,0.000000,17.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,2019.000000,6.000000,16.000000,1.000000,0.000000
75%,2020-03-25 20:08:00,22.000000,34.000000,989.110000,13.000000,0.000000,22.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,2020.000000,9.000000,23.000000,2.000000,0.000000
max,2021-01-01 07:53:00,46.000000,47.000000,999.650000,57.000000,18.540000,42.000000,5.000000,1.000000,1.000000,...,1.000000,1.000000,0.0,1.000000,1.000000,2021.000000,12.000000,31.000000,3.000000,1.000000
std,NaN,6.454604,5.400388,3.607065,6.699889,0.471682,6.321100,0.589723,0.066781,0.027418,...,0.025481,0.242821,0.0,0.173361,0.288623,0.811911,3.456638,8.789202,1.121245,0.348893


In [22]:
weather_2021.rename(columns={
    "time": "timestamp",
    "wspd": "windspeed",
    "pres": "pressure",
    "prcp": "precipitation",
    "temp": "temperature"
}, inplace=True)

In [23]:
# Änderungen überprüfen
weather_2021.head()

,timestamp,temperature,dwpt,rhum,precipitation,snow,wdir,windspeed,wpgt,pressure,tsun,coco
0,2021-01-01 00:00:00,10.0,-1.7,44,0.0,NaN,50,5.4,NaN,1015.0,NaN,NaN
1,2021-01-01 01:00:00,15.6,-5.5,23,0.0,NaN,50,11.2,NaN,1015.0,NaN,NaN
2,2021-01-01 02:00:00,15.6,-5.5,23,0.0,NaN,40,13.0,NaN,1015.2,NaN,NaN
3,2021-01-01 03:00:00,14.4,-5.4,25,0.0,NaN,360,18.4,NaN,1015.2,NaN,NaN
4,2021-01-01 04:00:00,15.0,-6.6,22,0.0,NaN,30,16.6,NaN,1015.3,NaN,NaN


In [24]:
weather_2021_filtered = weather_2021[["timestamp", "temperature", "windspeed", "pressure", "precipitation"]]

In [ ]:
combined_weather_set = pd.concat([weather_set, weather_2021_filtered], ignore_index=True)

In [26]:
combined_weather_set.head()

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature,temperature_difference,...,mostly,thunder_in_vicinity,light,partly,timestamp_year,timestamp_month,timestamp_day,timestamp_time,season,isWeekday
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0,1.0,...,0.0,0.0,0.0,0.0,2018.0,1.0,1.0,08:53:00,0.0,0.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,1.0,1.0,09:53:00,0.0,0.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,1.0,1.0,10:53:00,0.0,0.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,1.0,2018.0,1.0,1.0,11:53:00,0.0,0.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,2018.0,1.0,1.0,12:53:00,0.0,0.0


In [30]:
# combined_weather_set['timestamp'] = pd.to_datetime(combined_weather_set['timestamp'], errors='coerce')

In [33]:
# filtered_weather_set = combined_weather_set[combined_weather_set['timestamp'].dt.year > 2020]

In [ ]:
# filtered_weather_set

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature,temperature_difference,...,mostly,thunder_in_vicinity,light,partly,timestamp_year,timestamp_month,timestamp_day,timestamp_time,season,isWeekday
29236,Burbank,2021-01-01 00:53:00,14.0,34.0,Fair,985.49,20.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,2021.0,1.0,1.0,00:53:00,0.0,0.0
29237,Burbank,2021-01-01 01:53:00,14.0,33.0,Fair,986.15,22.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,2021.0,1.0,1.0,01:53:00,0.0,0.0
29238,Burbank,2021-01-01 02:53:00,14.0,33.0,Fair,986.48,19.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,2021.0,1.0,1.0,02:53:00,0.0,0.0
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,2021.0,1.0,1.0,03:53:00,0.0,0.0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,2021.0,1.0,1.0,04:53:00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37999,NaN,2021-12-31 19:00:00,12.2,NaN,NaN,1006.30,9.4,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38000,NaN,2021-12-31 20:00:00,11.7,NaN,NaN,1006.70,25.9,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38001,NaN,2021-12-31 21:00:00,11.1,NaN,NaN,1007.80,9.4,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38002,NaN,2021-12-31 22:00:00,11.1,NaN,NaN,1008.80,11.2,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
